# Programming assignment 5, module 1 (70 points)
(adapted from the work done by Erik Learned-Miller, which was originally developed by Fei-Fei Li, Andrej Karpathy, and Justin Johnson)

## Overview
<font size='4'> In this assignment you will practice putting together a Convolution Neural Network (CNN) classification pipeline. So far we have worked with deep fully-connected networks, using them to explore different optimization strategies and network architectures. Fully-connected networks are a good testbed for experimentation because they are very computationally efficient, but in practice all state-of-the-art results use convolutional networks instead.

<font size='4'>In this module, you will implement several layer types that are used in convolutional networks. You will then use these layers to train a convolutional network on the CIFAR-10 dataset.

## Submission format
* <font size='4'>`<your_nu_username>_cnn.ipynb`
    
## Note: 
* <font size='4'>Do not install any additional packages inside the conda environment. The TAs will use the same environment as defined in the config files we provide you, so anything that’s not in there by default will probably cause your code to break during grading. Failure to follow any of these instructions will lead to point deductions. 
* <font size='4'>We have some inline questions embedded in the Jupyter notebook files. Do not miss them.

## setup

In [1]:
# As usual, a bit of setup
from __future__ import print_function
import time
import numpy as np
import matplotlib.pyplot as plt
# from utils.classifiers.fc_net import *
from utils.data_utils import get_CIFAR10_data
from utils.gradient_check import eval_numerical_gradient, eval_numerical_gradient_array
from utils.solver import Solver
from utils.layers import affine_forward, affine_backward, relu_forward, relu_backward, softmax_loss

%matplotlib inline
plt.rcParams['figure.figsize'] = (10.0, 8.0) # set default size of plots
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'

# for auto-reloading external modules
# see http://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython
%load_ext autoreload
%autoreload 2

def rel_error(x, y):
    """ returns relative error """
    return np.max(np.abs(x - y) / (np.maximum(1e-8, np.abs(x) + np.abs(y))))

In [2]:
# let's download the data
%cd ../datasets

# 1 -- Linux 
# 2 -- MacOS
# 3 -- Command Prompt on Windows
# 4 -- manually downloading the data
choice = 3


if choice == 1:
    # should work well on Linux and in Powershell on Windows
    !wget http://www.cs.toronto.edu/~kriz/cifar-10-python.tar.gz
elif choice == 2 or choice ==3:
    # if wget is not available for you, try curl
    # should work well on MacOS
    !curl http://www.cs.toronto.edu/~kriz/cifar-10-python.tar.gz --output cifar-10-python.tar.gz
else:
    print('Please manually download the data from http://www.cs.toronto.edu/~kriz/cifar-10-python.tar.gz and put it under the datasets folder.')
!tar -xzvf cifar-10-python.tar.gz

if choice==3:
    !del cifar-10-python.tar.gz
else:
    !rm cifar-10-python.tar.gz

/Users/lukedavidson/Downloads/northeastern/cs5330_cv/pa5_1/datasets
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  162M  100  162M    0     0  21.3M      0  0:00:07  0:00:07 --:--:-- 25.5M
x cifar-10-batches-py/
x cifar-10-batches-py/data_batch_4
x cifar-10-batches-py/readme.html
x cifar-10-batches-py/test_batch
x cifar-10-batches-py/data_batch_3
x cifar-10-batches-py/batches.meta
x cifar-10-batches-py/data_batch_2
x cifar-10-batches-py/data_batch_5
x cifar-10-batches-py/data_batch_1
/bin/bash: del: command not found


In [3]:
# Load the (preprocessed) CIFAR10 data.
cifar10_dir = '../datasets/cifar-10-batches-py'

data = get_CIFAR10_data(cifar10_dir)
for k, v in list(data.items()):
    print(('%s: ' % k, v.shape))

('X_train: ', (49000, 3, 32, 32))
('y_train: ', (49000,))
('X_val: ', (1000, 3, 32, 32))
('y_val: ', (1000,))
('X_test: ', (1000, 3, 32, 32))
('y_test: ', (1000,))


## Part 1: Convolution layers (14 points)

<font size="4" color="red">**task 1.1: forward pass of a convolution layer (7 points)**

In [6]:
def conv_forward_naive(x, w, b, conv_param):
    """
    A naive implementation of the forward pass for a convolutional layer.

    The input consists of N data points, each with C channels, height H and
    width W. We convolve each input with F different filters, where each filter
    spans all C channels and has height HH and width WW.

    Input:
    - x: Input data of shape (N, C, H, W)
    - w: Filter weights of shape (F, C, HH, WW)
    - b: Biases, of shape (F,)
    - conv_param: A dictionary with the following keys:
      - 'stride': The number of pixels between adjacent receptive fields in the
        horizontal and vertical directions.
      - 'pad': The number of pixels that will be used to zero-pad the input.


    During padding, 'pad' zeros should be placed symmetrically (i.e equally on both sides)
    along the height and width axes of the input. Be careful not to modfiy the original
    input x directly.

    Returns a tuple of:
    - out: Output data, of shape (N, F, H', W') where H' and W' are given by
      H' = 1 + (H + 2 * pad - HH) / stride
      W' = 1 + (W + 2 * pad - WW) / stride
    - cache: (x, w, b, conv_param)
    """
    # Shape params
    N, C, H, W = x.shape
    F, C, fil_height, fil_width = w.shape
    stride = conv_param['stride']
    pad_val = conv_param['pad']
    new_H = H + 2*pad_val
    new_W = W + 2*pad_val
    
    # Padding
    x_new = np.pad(x, [(0,),(0,),(pad_val,),(pad_val,)])

    # H' = 1 + (H + 2 * pad - HH) / stride
    H_prime = int(1 + (x.shape[2] + (2*pad_val) - w.shape[2])/stride)
    
    # W' = 1 + (W + 2 * pad - WW) / stride
    W_prime = int(1 + (x.shape[3] + (2*pad_val) - w.shape[3])/stride)

    # NAIVE CONVOLUTION LAYERS
    out = np.empty((N, F, H_prime, W_prime))
    for im_num in range(N):
        for filter_num in range(F):
            for y_inc in range(H_prime):
                y_index_start = y_inc*stride
                y_index_end = y_index_start + fil_height
                for x_inc in range(W_prime):
                    x_index_start = x_inc*stride
                    x_index_end = x_index_start + fil_width
                    Wx_sum = np.sum(x_new[im_num, :, y_index_start:y_index_end, x_index_start:x_index_end] * w[filter_num, :, :, :])
                    bias = b[filter_num,]
                    out[im_num, filter_num, y_inc, x_inc] = Wx_sum + bias    # W@x + b
    
    cache = (x, w, b, conv_param)
    return out, cache

In [7]:
# check your forward pass implementation
x_shape = (2, 3, 4, 4)
w_shape = (3, 3, 4, 4)
x = np.linspace(-0.1, 0.5, num=np.prod(x_shape)).reshape(x_shape)
w = np.linspace(-0.2, 0.3, num=np.prod(w_shape)).reshape(w_shape)
b = np.linspace(-0.1, 0.2, num=3)

conv_param = {'stride': 2, 'pad': 1}
out, _ = conv_forward_naive(x, w, b, conv_param)
correct_out = np.array([[[[-0.08759809, -0.10987781],
                           [-0.18387192, -0.2109216 ]],
                          [[ 0.21027089,  0.21661097],
                           [ 0.22847626,  0.23004637]],
                          [[ 0.50813986,  0.54309974],
                           [ 0.64082444,  0.67101435]]],
                         [[[-0.98053589, -1.03143541],
                           [-1.19128892, -1.24695841]],
                          [[ 0.69108355,  0.66880383],
                           [ 0.59480972,  0.56776003]],
                          [[ 2.36270298,  2.36904306],
                           [ 2.38090835,  2.38247847]]]])

# Compare your output to ours; difference should be around e-8
print('Testing conv_forward_naive')
print('difference: ', rel_error(out, correct_out))

Testing conv_forward_naive
difference:  2.2121476417505994e-08


<font size="4" color="red">**task 1.2: backward pass of a convolution layer (7 points)**

In [10]:
def conv_backward_naive(dout, cache):
    """
    A naive implementation of the backward pass for a convolutional layer.

    Inputs:
    - dout: Upstream derivatives.
    - cache: A tuple of (x, w, b, conv_param) as in conv_forward_naive

    Returns a tuple of:
    - dx: Gradient with respect to x
    - dw: Gradient with respect to w
    - db: Gradient with respect to b
    """
    # Cache
    x, W, b, conv_param = cache

    # Shape params
    Nx, Cx, Hx, Wx = x.shape
    Fw, Cw, HHw, WWw = W.shape
    Nd, Fd, Hd, Wd = dout.shape    
    stride = conv_param['stride']
    pad = conv_param['pad']
    
    # initialize matrices
    dx = np.zeros(x.shape)                           
    dw = np.zeros(W.shape)
    db = np.empty((Fd,))
    
    # db
    for i in range(Fd):
        db[i,] = np.sum(dout[:,i,:,:])
    
    # Padding
    x_new = np.pad(x, ((0,),(0,), (pad,), (pad,)))
    dx_new = np.pad(dx, ((0,),(0,), (pad,), (pad,)))
    
    # Backprop process...
    for i in range(Nd):
        x_sub_new = x_new[i, :, :, :]    # (C, H+pad, W+pad)
        dx_sub_new = dx_new[i, :, :, :]    # (C, H+pad, W+pad)
        for j in range(Hd):
            if stride == 1:
                y_index_start = j*stride
                y_index_end = y_index_start + HHw
            else:
                y_index_start = j*stride
                y_index_end = y_index_start + HHw
            for k in range(Wd):
                if stride == 1:
                    x_index_start = k*stride
                    x_index_end = x_index_start + WWw
                else:
                    x_index_start = k*int(stride/2)
                    x_index_end = x_index_start + WWw
                for L in range(Fd):
                    x_sub_new_slice = x_sub_new[:, y_index_start:y_index_end, x_index_start:x_index_end]    # C, sub, sub
                    dx_sub_new[:, y_index_start:y_index_end, x_index_start:x_index_end] += W[L, :, :, :] * dout[i, L, j, k]
                    dw[L, :, :, :] += x_sub_new_slice * dout[i, L, j, k]
        dx[i, :, :, :] = dx_sub_new[:, pad:-pad, pad:-pad]
    
    return dx, dw, db

In [11]:
# gradient check
np.random.seed(231)
x = np.random.randn(4, 3, 5, 5)
w = np.random.randn(2, 3, 3, 3)
b = np.random.randn(2,)
dout = np.random.randn(4, 2, 5, 5)
conv_param = {'stride': 1, 'pad': 1}

dx_num = eval_numerical_gradient_array(lambda x: conv_forward_naive(x, w, b, conv_param)[0], x, dout)
dw_num = eval_numerical_gradient_array(lambda w: conv_forward_naive(x, w, b, conv_param)[0], w, dout)
db_num = eval_numerical_gradient_array(lambda b: conv_forward_naive(x, w, b, conv_param)[0], b, dout)

out, cache = conv_forward_naive(x, w, b, conv_param)
dx, dw, db = conv_backward_naive(dout, cache)

# Your errors should be around e-8 or less.
print('Testing conv_backward_naive function')
print('dx error: ', rel_error(dx, dx_num))
print('dw error: ', rel_error(dw, dw_num))
print('db error: ', rel_error(db, db_num))

Testing conv_backward_naive function
dx error:  1.1597815076211182e-08
dw error:  2.2471264748452487e-10
db error:  3.3726153958780465e-11


## Part 2: Spatial Batch Normalization (22 points)
<font size='4'>Batch normalization is a very useful technique for training deep neural networks. As proposed in the original paper [1], batch normalization can also be used for convolutional networks, but we need to tweak it a bit; the modification will be called "spatial batch normalization."

<font size='4'>Normally batch-normalization accepts inputs of shape `(N, D)` and produces outputs of shape `(N, D)`, where we normalize across the minibatch dimension `N`. For data coming from convolutional layers, batch normalization needs to accept inputs of shape `(N, C, H, W)` and produce outputs of shape `(N, C, H, W)` where the `N` dimension gives the minibatch size and the `(H, W)` dimensions give the spatial size of the feature map.

<font size='4'>If the feature map was produced using convolutions, then we expect the statistics of each feature channel to be relatively consistent both between different imagesand different locations within the same image. Therefore spatial batch normalization computes a mean and variance for each of the `C` feature channels by computing statistics over both the minibatch dimension `N` and the spatial dimensions `H` and `W`.


[1] [Sergey Ioffe and Christian Szegedy, "Batch Normalization: Accelerating Deep Network Training by Reducing
Internal Covariate Shift", ICML 2015.](https://arxiv.org/abs/1502.03167)

<font size='4' color='red'>**Task 2.1: forward pass of a (normal) batch norm layer (7 points).**

In [12]:
def batchnorm_forward(x, gamma, beta, bn_param):
    """
    Forward pass for batch normalization.

    During training the sample mean and (uncorrected) sample variance are
    computed from minibatch statistics and used to normalize the incoming data.
    During training we also keep an exponentially decaying running mean of the
    mean and variance of each feature, and these averages are used to normalize
    data at test-time.

    At each timestep we update the running averages for mean and variance using
    an exponential decay based on the momentum parameter:

    running_mean = momentum * running_mean + (1 - momentum) * sample_mean
    running_var = momentum * running_var + (1 - momentum) * sample_var

    Note that the batch normalization paper suggests a different test-time
    behavior: they compute sample mean and variance for each feature using a
    large number of training images rather than using a running average. For
    this implementation we have chosen to use running averages instead since
    they do not require an additional estimation step; the torch7
    implementation of batch normalization also uses running averages.

    Input:
    - x: Data of shape (N, D)
    - gamma: Scale parameter of shape (D,)
    - beta: Shift paremeter of shape (D,)
    - bn_param: Dictionary with the following keys:
      - mode: 'train' or 'test'; required
      - eps: Constant for numeric stability
      - momentum: Constant for running mean / variance.
      - running_mean: Array of shape (D,) giving running mean of features
      - running_var Array of shape (D,) giving running variance of features

    Returns a tuple of:
    - out: of shape (N, D)
    - cache: A tuple of values needed in the backward pass
    """
    # Params
    mode = bn_param['mode']
    eps = bn_param.get('eps', 1e-5)
    momentum = bn_param.get('momentum', 0.9)
    
    # Shape
    N, D = x.shape    
    running_mean = bn_param.get('running_mean', np.zeros(D, dtype=x.dtype))
    running_var = bn_param.get('running_var', np.zeros(D, dtype=x.dtype))

    if mode == 'train':
        # Calcs
        x_sum = np.sum(x, axis=0)
        mean = x_sum/x.shape[0]
        x_minus_mean = x - mean
        x_minus_mean_sq = x_minus_mean**2
        sigma_sq = (np.sum(x_minus_mean_sq, axis=0))/x.shape[0]
        denom = np.sqrt(sigma_sq + eps)
        
        # Normalize
        normalized_x = x_minus_mean/denom
        
        # out, running_mean, running_var, cache
        out = (normalized_x*gamma) + beta
        running_mean = momentum * running_mean + (1 - momentum) * mean
        running_var = momentum * running_var + (1 - momentum) * sigma_sq
        cache = (x, gamma, x_minus_mean, denom)
    elif mode == 'test':
        mean = running_mean
        sigma_sq = running_var
        
        # Calcs
        x_minus_mean = x - mean
        denom = np.sqrt(sigma_sq+eps)
        normalized_x = x_minus_mean/denom
        out = normalized_x*gamma + beta
    else:
        raise ValueError('Invalid forward batchnorm mode "%s"' % mode)

    # Store the updated running means back into bn_param
    bn_param['running_mean'] = running_mean
    bn_param['running_var'] = running_var

    return out, cache

<font size='4' color='red'>**Task 2.2: forward pass of a spatial batch norm layer (4 points).**

In [19]:
def spatial_batchnorm_forward(x, gamma, beta, bn_param):
    """
    Computes the forward pass for spatial batch normalization.

    Inputs:
    - x: Input data of shape (N, C, H, W)
    - gamma: Scale parameter, of shape (C,)
    - beta: Shift parameter, of shape (C,)
    - bn_param: Dictionary with the following keys:
      - mode: 'train' or 'test'; required
      - eps: Constant for numeric stability
      - momentum: Constant for running mean / variance. momentum=0 means that
        old information is discarded completely at every time step, while
        momentum=1 means that new information is never incorporated. The
        default of momentum=0.9 should work well in most situations.
      - running_mean: Array of shape (D,) giving running mean of features
      - running_var Array of shape (D,) giving running variance of features

    Returns a tuple of:
    - out: Output data, of shape (N, C, H, W)
    - cache: Values needed for the backward pass
    """
    # Shape params
    N, C, H, W = x.shape
                        # input shapes to 2.1: x = (N,D) ; gamma/beta = (D,)
                        # in this case, C = D, want C as last dim
    
    # Rearranging calcs
    x_rearrange_NHWC = np.transpose(x, axes=(0,2,3,1))    # arrange C as last dim
    x_dim = N*H*W
    x_reshape = x_rearrange_NHWC.reshape(x_dim, C)    # reshape in to (_,C) for input to 2.1

    out, cache = batchnorm_forward(x_reshape, gamma, beta, bn_param)
    
    out_reshape = out.reshape(N, H, W, C)   # re-separate data
    out = np.transpose(out_reshape, axes=(0,3,1,2))    # rearrange back to N, C, H, W
    

    return out, cache

In [20]:
np.random.seed(231)
# Check the training-time forward pass by checking means and variances
# of features both before and after spatial batch normalization

N, C, H, W = 2, 3, 4, 5
x = 4 * np.random.randn(N, C, H, W) + 10

print('Before spatial batch normalization:')
print('  Shape: ', x.shape)
print('  Means: ', x.mean(axis=(0, 2, 3)))
print('  Stds: ', x.std(axis=(0, 2, 3)))

# Means should be close to zero and stds close to one
gamma, beta = np.ones(C), np.zeros(C)
bn_param = {'mode': 'train'}
out, _ = spatial_batchnorm_forward(x, gamma, beta, bn_param)
print('After spatial batch normalization:')
print('  Shape: ', out.shape)
print('  Means: ', out.mean(axis=(0, 2, 3)))
print('  Stds: ', out.std(axis=(0, 2, 3)))

# Means should be close to beta and stds close to gamma
gamma, beta = np.asarray([3, 4, 5]), np.asarray([6, 7, 8])
out, _ = spatial_batchnorm_forward(x, gamma, beta, bn_param)
print('After spatial batch normalization (nontrivial gamma, beta):')
print('  Shape: ', out.shape)
print('  Means: ', out.mean(axis=(0, 2, 3)))
print('  Stds: ', out.std(axis=(0, 2, 3)))

Before spatial batch normalization:
  Shape:  (2, 3, 4, 5)
  Means:  [9.33463814 8.90909116 9.11056338]
  Stds:  [3.61447857 3.19347686 3.5168142 ]
After spatial batch normalization:
  Shape:  (2, 3, 4, 5)
  Means:  [ 6.18949336e-16  5.99520433e-16 -1.22124533e-16]
  Stds:  [0.99999962 0.99999951 0.9999996 ]
After spatial batch normalization (nontrivial gamma, beta):
  Shape:  (2, 3, 4, 5)
  Means:  [6. 7. 8.]
  Stds:  [2.99999885 3.99999804 4.99999798]


In [21]:
np.random.seed(231)
# Check the test-time forward pass by running the training-time
# forward pass many times to warm up the running averages, and then
# checking the means and variances of activations after a test-time
# forward pass.
N, C, H, W = 10, 4, 11, 12

bn_param = {'mode': 'train'}
gamma = np.ones(C)
beta = np.zeros(C)
for t in range(50):
  x = 2.3 * np.random.randn(N, C, H, W) + 13
  spatial_batchnorm_forward(x, gamma, beta, bn_param)
bn_param['mode'] = 'test'
x = 2.3 * np.random.randn(N, C, H, W) + 13
a_norm, _ = spatial_batchnorm_forward(x, gamma, beta, bn_param)

# Means should be close to zero and stds close to one, but will be
# noisier than training-time forward passes.
print('After spatial batch normalization (test-time):')
print('  means: ', a_norm.mean(axis=(0, 2, 3)))
print('  stds: ', a_norm.std(axis=(0, 2, 3)))

After spatial batch normalization (test-time):
  means:  [-0.08034406  0.07562881  0.05716371  0.04378383]
  stds:  [0.96718744 1.0299714  1.02887624 1.00585577]


<font size='4' color='red'>**Task 2.3: backward pass of a (normal) batch norm layer (7 points).**

In [22]:
def batchnorm_backward(dout, cache):
    """
    Backward pass for batch normalization.
    
    For this implementation you should work out the derivatives for the batch
    normalizaton backward pass on paper and simplify as much as possible. You
    should be able to derive a simple expression for the backward pass.

    Inputs:
    - dout: Upstream derivatives, of shape (N, D)
    - cache: Variable of intermediates from batchnorm_forward.

    Returns a tuple of:
    - dx: Gradient with respect to inputs x, of shape (N, D)
    - dgamma: Gradient with respect to scale parameter gamma, of shape (D,)
    - dbeta: Gradient with respect to shift parameter beta, of shape (D,)
    """
    # Cache
    x, gamma, x_minus_mean, denom = cache
    
    # Shapes
    N,D = x.shape
    
    # Grads
    dgamma = np.sum(((x_minus_mean/denom)*dout), axis=0)
    dbeta = np.sum(dout, axis=0)
    dx = (gamma/denom/N)*(N*dout-dbeta-((x_minus_mean/(denom**2))*np.sum((dout*x_minus_mean),axis=0)))

    return dx, dgamma, dbeta

<font size='4' color='red'>**Task 2.4: backward pass of a spatial batch norm layer (4 points).**

In [23]:
def spatial_batchnorm_backward(dout, cache):
    """
    Computes the backward pass for spatial batch normalization.

    Inputs:
    - dout: Upstream derivatives, of shape (N, C, H, W)
    - cache: Values from the forward pass

    Returns a tuple of:
    - dx: Gradient with respect to inputs, of shape (N, C, H, W)
    - dgamma: Gradient with respect to scale parameter, of shape (C,)
    - dbeta: Gradient with respect to shift parameter, of shape (C,)
    """
    # Shapes
    N,C,H,W = dout.shape
                        # batchnorm_backward inputs = dout (N,D), cache
                        # D serving as C
    
    # Rearranging Calcs
    dout_rearrange_NHWC = np.transpose(dout, axes=(0, 2, 3, 1))    # arrange C as last dim
    x_dim = N*H*W
    dout_reshape = dout_rearrange_NHWC.reshape(x_dim, C)    # reshape in to (_,C) for input to 2.3
    
    # Grads
    dx, dgamma, dbeta = batchnorm_backward(dout_reshape, cache)
    dx_reshape = dx.reshape(N, H, W, C)   # re-separate data
    dx = np.transpose(dx_reshape, axes=(0,3,1,2))    # rearrange back to N, C, H, W
    return dx, dgamma, dbeta

In [24]:
# gradient check
np.random.seed(231)
N, C, H, W = 2, 3, 4, 5
x = 5 * np.random.randn(N, C, H, W) + 12
gamma = np.random.randn(C)
beta = np.random.randn(C)
dout = np.random.randn(N, C, H, W)

bn_param = {'mode': 'train'}
fx = lambda x: spatial_batchnorm_forward(x, gamma, beta, bn_param)[0]
fg = lambda a: spatial_batchnorm_forward(x, gamma, beta, bn_param)[0]
fb = lambda b: spatial_batchnorm_forward(x, gamma, beta, bn_param)[0]

dx_num = eval_numerical_gradient_array(fx, x, dout)
da_num = eval_numerical_gradient_array(fg, gamma, dout)
db_num = eval_numerical_gradient_array(fb, beta, dout)

#You should expect errors of magnitudes between 1e-12~1e-06
_, cache = spatial_batchnorm_forward(x, gamma, beta, bn_param)
dx, dgamma, dbeta = spatial_batchnorm_backward(dout, cache)
print('dx error: ', rel_error(dx_num, dx))
print('dgamma error: ', rel_error(da_num, dgamma))
print('dbeta error: ', rel_error(db_num, dbeta))

dx error:  2.786648191930665e-07
dgamma error:  7.0974817113608705e-12
dbeta error:  3.275608725278405e-12


## Part 3: Adaptive average pooling layer (8 points)
<font size='4'> In AlexNet and VGG-like networks, a 2D convolution feature map is usually flattened to get a 1D feature vector, which is then fed into a fully-connected layer. Since ResNet, such flattening is no longer used. Instead, an adaptive average pooing layer is used. Given a 2D feature map with shape of `(N, C, H, W)`, the mean across the dimension `H` and `W` are computed. As a result, we get a 2D feature map with shape of `(N, C, 1, 1)` that is equivalent to a 1D feature vector with shape of `(N, C)`.

<font size='4' color='red'>**Task 3.1: forward pass of an adaptive average pooling layer (4 points).**

In [25]:
def adaptive_avg_pool_forward(x):
    """
    Computes the forward pass of the adaptive average pooling layer
    
    Input:
    - x: Input data of shape (N, C, H, W)
    
    Returns of a tuple of:
    - out: Output data, of shape (N, C)
    - cache: (x,)
    """
    out, cache = None, (x)

    # avg_pool_forward
    out = np.mean(x, axis = (2,3))
    
    return out, cache

In [26]:
# check your implementation
x_shape = (2, 3, 4, 4)
x = np.linspace(-0.1, 0.5, num=np.prod(x_shape)).reshape(x_shape)

out, _ = adaptive_avg_pool_forward(x)
correct_out = np.array([
    [-0.05263158,  0.04842105,  0.14947368],
    [ 0.25052632,  0.35157895,  0.45263158]
])

# Compare your output to ours; difference should be around e-8
print('Testing adaptive_avg_pool_forward')
print('difference: ', rel_error(out, correct_out))

Testing adaptive_avg_pool_forward
difference:  2.7173913643149127e-08


In [28]:
def adaptive_avg_pool_backward(dout, cache):
    """
    Computes the forward pass of the adaptive average pooling layer
    
    Inputs:
    - dout: Upstream derivatives.
    - cache: x as in global_avg_pool_backward
    
    Returns:
    - dx: gradient with respect x
    
    """
    # Cache and shapes
    x = cache
    N, C, H, W = x.shape
    
    # Initialize and calc dx
    dx = np.zeros((N, C, H, W))
    dout_avg = dout/(H*W)    
    for i in range(N):
        for j in range(C):
            dx[i,j,:,:] = dout_avg[i,j]
            
    return dx    

In [29]:
# gradient check
np.random.seed(231)
x = np.random.randn(4, 3, 5, 5)
dout = np.random.randn(4, 3)

dx_num = eval_numerical_gradient_array(lambda x: adaptive_avg_pool_forward(x)[0], x, dout)

out, cache = adaptive_avg_pool_forward(x)
dx = adaptive_avg_pool_backward(dout, cache)

# Your errors should be around e-8 or less.
print('Testing adaptive_avg_pool_backward function')
print('dx error: ', rel_error(dx, dx_num))

Testing adaptive_avg_pool_backward function
dx error:  5.501115761541525e-11


## Part 4: ConvNet (26 points)
<font size='4'>Now that you have implemented all the necessary layers, we can put them together into a simple convolutional network.

<font size='4' color='red'>**Task 4.1: Implement a CNN (14 points).**

In [34]:
class ConvNet(object):
    """
    A simple convolutional network with the following architecture:

    [conv - bn - relu] x M - adaptive_average_pooling - affine - softmax
    
    "[conv - bn - relu] x M" means the "conv-bn-relu" architecture is repeated for
    M times, where M is implicitly defined by the convolution layers' parameters.
    
    For each convolution layer, we do downsampling of factor 2 by setting the stride
    to be 2. So we can have a large receptive field size.

    The network operates on minibatches of data that have shape (N, C, H, W)
    consisting of N images, each with height H and width W and with C input
    channels.
    """
    
    def __init__(self, input_dim=(3, 32, 32), num_filters=[32], filter_sizes=[7],
            num_classes=10, weight_scale=1e-3, reg=0.0, use_batch_norm=True, 
            dtype=np.float32):
        """
        Initialize a new network.

        Inputs:
        - input_dim: Tuple (C, H, W) giving size of input data
        - num_filters: Number of filters to use in the convolutional layer. It is a
          list whose length defines the number of convolution layers
        - filter_sizes: Width/height of filters to use in the convolutional layer. It
          is a list with the same length with num_filters
        - num_classes: Number of output classes
        - weight_scale: Scalar giving standard deviation for random initialization
          of weights.
        - reg: Scalar giving L2 regularization strength
        - use_batch_norm: A boolean variable indicating whether to use batch normalization
        - dtype: numpy datatype to use for computation.
        """
        self.params = {}
        self.reg = reg
        self.dtype = dtype
        
        # Assertion check
        assert len(num_filters) == len(filter_sizes)

        # Params and shapes
        self.filter_sizes = filter_sizes
        self.num_filters = num_filters
        self.use_batch_norm = use_batch_norm
        M = len(num_filters)
        C, H, W = input_dim
        
        # Init W's and b's
        for i in range(M+1):
            if i == 0:
                F = num_filters[i]
                HH_WW = filter_sizes[i]
                if use_batch_norm == True:
                    W_id = 'W' + str(i+1)
                    b_id = 'b' + str(i+1)
                    gamma_id = 'gamma' + str(i+1)
                    beta_id = 'beta' + str(i+1)
                        # W = (F,C,HH,WW)
                    self.params[W_id] = weight_scale*np.random.randn(F,C,HH_WW,HH_WW)
                        # b = (F,)
                    self.params[b_id] = np.zeros(F)
                        # gamma = (F,)
                    self.params[gamma_id] = np.ones(F)
                        # beta = (F,)
                    self.params[beta_id] = np.zeros(F)
                elif use_batch_norm == False:
                    W_id = 'W' + str(i+1)
                    b_id = 'b' + str(i+1)
                    self.params[W_id] = weight_scale*np.random.randn(F, C, HH_WW, HH_WW)
                    self.params[b_id] = np.zeros(F)
                else:
                    raise NotImplementedError('Invalid use_batch_norm value.')
            elif i == M:
                if use_batch_norm == True:
                    W_id = 'W' + str(i+1)
                    b_id = 'b' + str(i+1)
                        # W = (F_last,Classes)
                    self.params[W_id] = weight_scale*np.random.randn(F, num_classes)
                        # b = (Classes,)
                    self.params[b_id] = np.zeros(num_classes)
                elif use_batch_norm == False:
                    W_id = 'W' + str(i+1)
                    b_id = 'b' + str(i+1)
                    self.params[W_id] = weight_scale*np.random.randn(F, num_classes)
                    self.params[b_id] = np.zeros(num_classes)
                else:
                    raise NotImplementedError('Invalid use_batch_norm value.')
            else:
                F = num_filters[i]
                F_last = num_filters[i-1]
                HH_WW = filter_sizes[i]
                if use_batch_norm == True:
                    W_id = 'W' + str(i+1)
                    b_id = 'b' + str(i+1)
                    gamma_id = 'gamma' + str(i+1)
                    beta_id = 'beta' + str(i+1)
                        # W = (F,F_last,HH,WW)
                    self.params[W_id] = weight_scale*np.random.randn(F, F_last, HH_WW, HH_WW)
                        # b = (F,)
                    self.params[b_id] = np.zeros(F)
                        # gamma = (F,)
                    self.params[gamma_id] = np.ones(F)
                        # beta = (F,)
                    self.params[beta_id] = np.zeros(F)
                elif use_batch_norm == False:
                    W_id = 'W' + str(i+1)
                    b_id = 'b' + str(i+1)
                    self.params[W_id] = weight_scale*np.random.randn(F, F_last, HH_WW, HH_WW)
                    self.params[b_id] = np.zeros(F)
                else:
                    raise NotImplementedError('Invalid use_batch_norm value.')

        for k, v in self.params.items():
            self.params[k] = v.astype(dtype)


    def loss(self, X, y=None):
        """
        Evaluate loss and gradient for the three-layer convolutional network.

        Input / output: Same API as TwoLayerNet in fc_net.py.
        
-------------------------------------------------- FROM PA4 --------------------------------------------------
        Inputs:
        - X: Array of input data of shape (N, d_1, ..., d_k)
        - y: Array of labels, of shape (N,). y[i] gives the label for X[i].
        
        Returns:
        If y is None, then run a test-time forward pass of the model and return:
        - scores: Array of shape (N, C) giving classification scores, where
          scores[i, c] is the classification score for X[i] and class c.
        If y is not None, then run a training-time forward and backward pass and
        return a tuple of:
        - loss: Scalar value giving the loss
        - grads: Dictionary with the same keys as self.params, mapping parameter
          names to gradients of the loss with respect to those parameters.
-------------------------------------------------- FROM PA4 --------------------------------------------------
      
        """
        # Init
        scores = None
        mode = 'test' if y is None else 'train'
        cache = []
        
        # Params and sizes
        filter_sizes = self.filter_sizes
        num_filters = self.num_filters
        use_batch_norm = self.use_batch_norm        
        conv_param['stride'] = 2
        M = len(num_filters)
        out_relu = X
        
        # Convolve
        if use_batch_norm == True:
            for i in range(M):
                conv_param['pad'] = int((filter_sizes[i]-1)/2)
                W_id = self.params['W'+str(i+1)]
                b_id = self.params['b'+str(i+1)]
                gamma_id = self.params['gamma'+str(i+1)]
                beta_id = self.params['beta'+str(i+1)]
                # convolve
                out_convolve, cache_convolve = conv_forward_naive(out_relu, W_id, b_id, conv_param)
                # batch normalization
                del bn_param['running_mean']
                del bn_param['running_var']
        
                out_bn, cache_bn = spatial_batchnorm_forward(out_convolve, gamma_id, beta_id, bn_param)   # need to pass one fix
                # relu
                out_relu, cache_relu = relu_forward(out_bn)
                cache.append((cache_convolve, cache_bn, cache_relu))
        else:
            for i in range(M):
                conv_param['pad'] = int((filter_sizes[i]-1)/2)
                W_id = self.params['W'+str(i+1)]
                b_id = self.params['b'+str(i+1)]
                gamma_id = self.params['gamma'+str(i+1)]
                beta_id = self.params['beta'+str(i+1)]
                # convolve
                out_convolve, cache_convolve = conv_forward_naive(out_relu, W_id, b_id, conv_param)
                # relu
                out_relu, cache_relu = relu_forward(out_convolve)
                cache.append((cache_convolve, cache_relu))
        
        # adaptive_average_pooling
        out_avgpool, cache_avgpool = adaptive_avg_pool_forward(out_relu)
        
        # affine
        W_id = self.params['W'+str(M+1)]
        b_id = self.params['b'+str(M+1)]
        scores, cache_affine = affine_forward(out_avgpool,W_id,b_id)

        if y is None:
            return scores

        loss, grads = 0, {}
        reg = self.reg
        
        # softmax
        loss, dout = softmax_loss(scores, y)
        
        # first layer
        # affine backwards
        W_id = 'W' + str(M+1)
        b_id = 'b' + str(M+1)
        dx_affine, grads[W_id], grads[b_id] = affine_backward(dout,cache_affine)
        loss += 0.5*reg*np.sum(self.params[W_id]**2)
        grads[W_id] += reg*self.params[W_id]
        
        # adaptive average backwards
        dout_avgpool = adaptive_avg_pool_backward(dx_affine, cache_avgpool)

        # Backprop
        for i in reversed(range(M)):
#             print(i)
            if use_batch_norm == True:
                W_id = 'W' + str(i+1)
                b_id = 'b' + str(i+1)
                gamma_id = 'gamma' + str(i+1)
                beta_id = 'beta' + str(i+1)
                cache_convolve,cache_bn,cache_relu = cache[i]
                    # relu back
                if i+1 == M:
                    dout_conv = dout_avgpool
                dout_relu = relu_backward(dout_conv, cache_relu)
                    # spatial back
                dx_spatial_back, grads[gamma_id], grads[beta_id] = spatial_batchnorm_backward(dout_relu, cache_bn)
                    # conv back
                dout_conv, grads[W_id], grads[b_id] = conv_backward_naive(dx_spatial_back, cache_convolve)
                loss += 0.5*reg*np.sum(self.params[W_id]**2)
                grads[W_id] += reg*self.params[W_id]
            else:
                W_id = 'W' + str(i+1)
                b_id = 'b' + str(i+1)
                cache_convolve,cache_relu = cache[i]
                    # relu back
                dout_relu = relu_backward(dout_avgpool, cache_relu)
                    # conv back
                dout_conv, grads[W_id], grads[b_id] = conv_backward_naive(dout_relu, cache_convolve)
                loss += 0.5*reg*np.sum(self.params[W_id]**2)
                grads[W_id] += reg*self.params[W_id]

        return loss, grads


### Sanity check loss
<font size='4'>After you build a new network, one of the first things you should do is sanity check the loss. When we use the softmax loss, we expect the loss for random weights (and no regularization) to be about `log(C)` for `C` classes. When we add regularization this should go up.

In [35]:
model = ConvNet()

N = 50
X = np.random.randn(N, 3, 32, 32)
y = np.random.randint(10, size=N)

loss, grads = model.loss(X, y)
print('Initial loss (no regularization): ', loss)
print('log(10): ', np.log(10))

model.reg = 0.5
loss, grads = model.loss(X, y)
print('Initial loss (with regularization): ', loss)

Initial loss (no regularization):  2.3024643397782953
log(10):  2.302585092994046
Initial loss (with regularization):  2.3036795070273244


### Gradient check
<font size='4'>After the loss looks reasonable, use numeric gradient checking to make sure that your backward pass is correct. When you use numeric gradient checking you should use a small amount of artifical data and a small number of neurons at each layer. Note: correct implementations may still have relative errors up to the order of e-2.

In [36]:
num_inputs = 2
input_dim = (3, 8, 8)
reg = 0.0
num_classes = 10
np.random.seed(231)
X = np.random.randn(num_inputs, *input_dim)
y = np.random.randint(num_classes, size=num_inputs)

model = ConvNet(input_dim=input_dim, dtype=np.float64)
loss, grads = model.loss(X, y)
# Errors should be small, but correct implementations may have
# relative errors up to the order of e-2
for param_name in sorted(grads):
    f = lambda _: model.loss(X, y)[0]
    param_grad_num = eval_numerical_gradient(f, model.params[param_name], verbose=False, h=1e-6)
    e = rel_error(param_grad_num, grads[param_name])
    print('%s max relative error: %e' % (param_name, rel_error(param_grad_num, grads[param_name])))

W1 max relative error: 1.000000e+00
W2 max relative error: 4.277113e-08
b1 max relative error: 2.602085e-09
b2 max relative error: 1.296123e-09
beta1 max relative error: 4.594574e-06
gamma1 max relative error: 5.442050e-06


### Overfit small data
<font size='4'>A nice trick is to train your model with just a few training samples. You should be able to overfit small datasets, which will result in high training accuracy and comparatively low validation accuracy.

In [33]:
np.random.seed(231)

num_train = 100
small_data = {
  'X_train': data['X_train'][:num_train],
  'y_train': data['y_train'][:num_train],
  'X_val': data['X_val'],
  'y_val': data['y_val'],
}

model = ConvNet(
    num_filters=[16, 32],
    filter_sizes=[7, 3],
    weight_scale=1.5e-2
)

solver = Solver(
    model, small_data,
    num_epochs=50, batch_size=20,
    update_rule='sgd_momentum',
    optim_config={
      'learning_rate': 1e-3,
    },
    verbose=True, print_every=10
)
solver.train()

ValueError: operands could not be broadcast together with shapes (3,6,7) (3,7,7) (3,6,7) 

In [ ]:
# Plotting the loss, training accuracy, and validation accuracy should show clear overfitting:
plt.subplot(2, 1, 1)
plt.plot(solver.loss_history, 'o')
plt.xlabel('iteration')
plt.ylabel('loss')

plt.subplot(2, 1, 2)
plt.plot(solver.train_acc_history, '-o')
plt.plot(solver.val_acc_history, '-o')
plt.legend(['train', 'val'], loc='upper left')
plt.xlabel('epoch')
plt.ylabel('accuracy')
plt.show()

<font size='4' color='red'>**4.2: Train a good CNN (7 points).**
    
<font size='4'>By tweaking different parameters, such as number of convolution layers, learning rate, batch size, etc, you should achieve greater than 62% accuracy on the validation set **with 3 epochs using the sgd_momentum optimizer**.
    
<font size='4'>If you are really careful, you should be able to get nearly 66% accuracy on the validation set. But we don't give extra credits for doing so.
    
<font size='4'>It may take a quite while for your training to be finished. **Do not use more than four convolution layers. Your training shouldn't be longer than one hour.** (This is a rough reference as it depends on the hardware. Our implementation takes less than 10 minutes to finish.)
    
<font size='4'>Use a large filter/kernel size in the first convolution layer (for example, 7), so you can easily visualize the learend filters.
    
<font size='4'>Since it is relatively slower to train a CNN, you can simply report the best hyper parameters you found. You need report validation accuracy of other choices below.

In [22]:
###### best_model = None
################################################################################
# TODO: Train the best ConvNet that you can on CIFAR-10 with 3 epochs using    #
# the sgd_momentum optimizer. Store your best model in the best_model variable.#
################################################################################

num_train = 100
small_data = {
  'X_train': data['X_train'][:num_train],
  'y_train': data['y_train'][:num_train],
  'X_val': data['X_val'],
  'y_val': data['y_val'],
}

model = ConvNet(
    num_filters=[32, 32, 16],
    filter_sizes=[7, 5, 3],
    weight_scale=1e-2
)

solver = Solver(
    model, small_data,
    num_epochs=3, batch_size=100,
    update_rule='sgd_momentum',
    optim_config={
      'learning_rate': 1e-2,
    },
    verbose=True, print_every=1
)
solver.train()

model = ConvNet(
    num_filters=[32, 32, 16, 8],
    filter_sizes=[7, 5, 3, 3],
    weight_scale=1e-2
)

best_model = model

# raise NotImplementedError

################################################################################
#                              END OF YOUR CODE                                #
################################################################################



(Iteration 1 / 3) loss: 2.305122
(Epoch 1 / 3) train acc: 0.120000; val_acc: 0.110000
(Iteration 2 / 3) loss: 2.304140
(Epoch 2 / 3) train acc: 0.120000; val_acc: 0.109000
(Iteration 3 / 3) loss: 2.301193
(Epoch 3 / 3) train acc: 0.120000; val_acc: 0.116000


In [23]:
# Run your best model on the validation and test sets. You should achieve above 62% accuracy on the validation set.
y_test_pred = np.argmax(best_model.loss(data['X_test']), axis=1)
y_val_pred = np.argmax(best_model.loss(data['X_val']), axis=1)
print('Validation set accuracy: ', (y_val_pred == data['y_val']).mean())
print('Test set accuracy: ', (y_test_pred == data['y_test']).mean())

KeyboardInterrupt: 

### Visualize Filters

In [ ]:
# You can visualize the first-layer convolutional filters from the trained network by running the following:
from utils.vis_utils import visualize_grid

grid = visualize_grid(model.params['W_1'].transpose(0, 2, 3, 1))
plt.imshow(grid.astype('uint8'))
plt.axis('off')
plt.gcf().set_size_inches(5, 5)
plt.show()

<font size='4' color='red'>**Task 4.3: report validation accuray for other hyper parameters you have tried (2 points)**

1. model = ConvNet(
    num_filters=[32, 32, 16, 8],
    filter_sizes=[7, 5, 3, 3],
    weight_scale=1e-2
)

solver = Solver(
    model, small_data,
    num_epochs=3, batch_size=100,
    update_rule='sgd_momentum',
    optim_config={
      'learning_rate': 1.8e-3,
    },
    verbose=True, print_every=1
)
Val acc: 0.073
Training acc: 0.060

2. model = ConvNet(
    num_filters=[32, 16, 8],
    filter_sizes=[7, 5, 3],
    weight_scale=1e-2
)

solver = Solver(
    model, small_data,
    num_epochs=3, batch_size=100,
    update_rule='sgd_momentum',
    optim_config={
      'learning_rate': 1e-3,
    },
    verbose=True, print_every=1
)
Val acc: 0.115
Training acc: 0.12

3. model = ConvNet(
    num_filters=[16, 32, 16],
    filter_sizes=[7, 5, 5],
    weight_scale=1e-3
)

solver = Solver(
    model, small_data,
    num_epochs=3, batch_size=100,
    update_rule='sgd_momentum',
    optim_config={
      'learning_rate': 1e-3,
    },
    verbose=True, print_every=1
)
Val acc: 0.070
Training acc: 0.098

4. model = ConvNet(
    num_filters=[16, 32, 16],
    filter_sizes=[7, 5, 5],
    weight_scale=1e-1
)

solver = Solver(
    model, small_data,
    num_epochs=3, batch_size=100,
    update_rule='sgd_momentum',
    optim_config={
      'learning_rate': 1e-2,
    },
    verbose=True, print_every=1
)
Val acc: 0.070
Training acc: 0.060

5. model = ConvNet(
    num_filters=[32, 32, 16],
    filter_sizes=[7, 5, 3],
    weight_scale=1e-2
)

solver = Solver(
    model, small_data,
    num_epochs=3, batch_size=100,
    update_rule='sgd_momentum',
    optim_config={
      'learning_rate': 1e-2,
    },
    verbose=True, print_every=1
)
Val acc: 0.116
Training acc: 0.11

<font size='4' color='red'>**Task 4.4: train a ConvNet without using batch normalization layers (3 points).**
    
<font size='4'>Report the best validation accuracy you can get and discuss how it is different from the version with batch normalization layers.

Validation accuracy is expected to be lower when use_batch_norm is set to false because the data between layers is not normalized to a unit factor.  This causes there to be more discrepensies within the data and for those discrepensies to continue and build throughout the neural network.